# Advanced GIS Term Paper: Wild Boar Risk for Vineyards

## Part 2: Cost Distance


In [2]:
import arcpy
from arcpy import env
from arcpy.sa import *

In [7]:
# set environment extent to mapping grid
arcpy.env.extent = arcpy.Raster("Suitability_combined").extent
print(arcpy.env.extent)

333200,000629869 5504499,96351364 342975,000629869 5519269,96351364 NaN NaN NaN NaN


#### Visibility
Wild Boar likes to move in Areas of low visibility <br>

In [3]:
nDSM = arcpy.Raster("DSM") - arcpy.Raster("DEM_5m_update.tif")

In [12]:
nDSM.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\nDSM")

Wie Sichtbarkeit ermitteln?
Jeder Pixel muss eine Arte Sichtbarkeit erhalten
Idee: Gridded Points und dazu Viewshed erstellen
Resample Raster --> Raster to Points --> Viewshed
Problem: Betrachter Punkte liegen auf dem Surface Raster, also auch auf einem Baum
Die Punkte müssen also auf DEM Level gesenkt werden
Idee: Punkte erstellen und Pixel wo Punkte liegen Wert von DEM einsetzen
Gibt Literatur Hinweis wie weit die Punke auseinander liegen müssten? Nein.
Hier is das Problem, dass der Betrachter, wenn er im Wald steht gar nichts sehen kann, da alle Pixel um ihn herum zu hoch sind.

DSM has 5m resolution --> would mean ~6 million points --> too much calculation for viewshed
Reduction of data: Exclude some areas where points can be because we know the visibility

Tool to create neighborhood statistics and smooth with different kernels: https://pro.arcgis.com/de/pro-app/tool-reference/spatial-analyst/focal-statistics.htm

Solution was to use point of view of boar and describe visibility by neighborhood statistic of every pixel


In [2]:
# Calculating NDVI
# Downloaded and imported Sentinel 2 data from September 2018
# Projected and clipped manually with geoprocessing tools
b4 = arcpy.Raster("B04_UTM32_Clip")
b8 = arcpy.Raster("B08_UTM32_Clip")
NDVI_sep18 = ((b8 - b4)/(b8 + b4))

In [3]:
NDVI_sep18.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\NDVI_sep18")

#### Looking for options to eliminate elec lines and smooth image to get protection of visibility

In [8]:
#Trying to find the elictricity lines (and other non vegetation tall objects)
Elec = (NDVI_sep18 > 0.5) & (arcpy.Raster("nDSM") > 20)

In [11]:
# Trying to eliminate electric lines (tall buildings)
nDSM = arcpy.Raster("nDSM")
nDSM_h35 = Con( (nDSM > 35) , 0.5, nDSM)

In [6]:
# just smoothing to get neighborhood values
nDSM_rad4_median = FocalStatistics("nDSM", NbrCircle(4, "CELL"), 
                               "PERCENTILE", "DATA", percentile_value = 50)

In [7]:
nDSM_rad7_median = FocalStatistics("nDSM", NbrCircle(7, "CELL"), 
                               "PERCENTILE", "DATA", percentile_value = 50)

In [8]:
nDSM_rad7_median.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\nDSM_rad7_median")

In [34]:
# Trying filtering the nDSM
nDSM_rad7_percentile25 = FocalStatistics("nDSM", NbrCircle(7, "CELL"), 
                               "PERCENTILE", "DATA", percentile_value = 25)

In [18]:
# Trying filtering the nDSM
nDSM_rect5_percentile25 = FocalStatistics("nDSM", NbrRectangle(5,5, "CELL"), 
                               "PERCENTILE", "DATA", percentile_value = 25)

In [35]:
# Trying filtering the nDSM
nDSM_annulus26_percentile20 = FocalStatistics("nDSM", NbrAnnulus(2,6, "CELL"), 
                               "PERCENTILE", "DATA", percentile_value = 20)

In [ ]:
# Trying to find edges
nDSM_edges_radius3 = FocalStatistics("nDSM", NbrCircle(3, "CELL"), 
                               "Range", "DATA")

In [30]:
# nDSM_annulus24_percentile10 looks like a good smoothing
nDSM_annulus26_percentile20.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\nDSM_annulus26_percentile20")

In [37]:
# nDSM_rad7_percentile25 looks better
nDSM_rad7_percentile25.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\nDSM_rad7_percentile25")

In [36]:
## Smoothing des elec-befreiten Bildes
nDSM_ann26_per20_mean5 = FocalStatistics("nDSM_annulus26_percentile20", NbrCircle(5, "CELL"), 
                               "MEAN", "DATA")

### Filtering of Elec lines and creating visibility raster
decided against manuell filtering of elec lines
tried different methods: annulus26_percentile20-filter worked well in eliminating the elec lines
also tried identifying elec lines, so they can be dealt with. Did not work well enough
tried identifying edge by compute range of moving window. In Theory they would have had high ranges, but it didnt work well enough
show edges map (nDSM_edges_radius3): Was one approach to identify lines. Could be continued by using annulus_percentile_filter only on smoothing areas with edges. This idea was not followed since the smoothing of the whole area is beneficial for getting a sense of visibility
I did not want to smooth the edges of the elec lines, i wanted to eliminate them. With smoothing the pixels would still have a high height and would be considered a beneficial visibility, but they should have no effect.
Disadvantage: Small strips of Trees or brushes also get eliminated
Result is very close to the corine landcover data of areas for woods

In [38]:
# upper limit
VisProtection_nb4 = arcpy.Raster("nDSM_rad7_percentile25")
VisProtection_nb4 = Con( (VisProtection_nb4 > 4) , 4, VisProtection_nb4)

In [40]:
# lower limit
VisProtection_nb02 = Con( (VisProtection_nb4 < 0.2) , 0.2, VisProtection_nb4)

In [41]:
VisProtection_nb02.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\VisProtection_nb02")

In [26]:
# Trying Heights + NDVI to only have high areas that are vegetation
VisProtection02_veg02 = Con( ((arcpy.Raster("VisProtection_nb02") > 0.2) & (arcpy.Raster("NDVI_sep18") > 0.2)) , 
                            arcpy.Raster("VisProtection_nb02"), 0)

In [41]:
VisProtection02_veg02.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\VisProtection02_veg02")

In [30]:
# Checking  low height areas with high ndvi
lowHeight01_highNDVI06 = Con( ((arcpy.Raster("nDSM_rad7_percentile25") < 0.1)  & (arcpy.Raster("NDVI_sep18") > 0.6)) , 
                            arcpy.Raster("NDVI_sep18"), "")

# Cost Factors
Visibility, Land cover class, NDVI <br>
NDVI thought about using NDVI as cost factor (boar likes to walk in vegetated areas) but low agrag field have high NDVI and would distort result to much

#### Visibility Cost

In [31]:
Vis_cost = 1 - (arcpy.Raster("VisProtection02_veg02")/4)

In [40]:
Vis_cost.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\Vis_cost")

In [2]:
Vis_cost_x9p1 = arcpy.Raster("Vis_cost")*9 + 1

In [3]:
Vis_cost_x9p1.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\Vis_cost_x9p1")

#### Land Cover Cost

In [6]:
# creating raster version of land cover classes
Corine_raster = arcpy.PolygonToRaster_conversion("CLC5_2018_Edit_Project", "Klassenname", 
                                 "Corine_raster", "MAXIMUM_AREA", cellsize = 5)

In [9]:
Corine_raster_CLC18 = arcpy.PolygonToRaster_conversion("CLC5_2018_Edit_Project", "CLC18", 
                                 "Corine_raster_CLC18", "MAXIMUM_AREA", cellsize = 5)

#### Costs for land cover types:
Discontinuous urban fabric - 1
Industrial or commercial units - 1
Road and rail networks and associated land - 0 (dealt with Atkis roads)
Mineral extraction sites - 0.8
Dump sites - 0.8
Construction sites - 1
Green urban areas - 0.7
Sport and leisure facilities - 0.7
Non-irrigated arable land - 0
Vineyards - 0
Fruit trees and berry plantations - 0
Pastures - 0
Broad-leaved forest - 0
3.1.2 Coniferous forest - 0
3.1.3 Mixed forest - 0 
3.2.1 Natural grassland - 0 
3.2.4 Transitional woodland/shrub - 0 
4.1.1 Inland marshes - 0.6
5.1.1 Water courses - 1
5.1.2 Water bodies - 1
Abandoned Vineyard - 0

In [29]:
# reclassify corine raster
# values from above but ranging from 0 to 10
Corine_raster_cost = Reclassify("Corine_raster", "Value", 
                         RemapValue([[1,10],[2,10],[3,0],[4,8],[5,8],[6,10],[7,7],
                                     [8,7],[9,0],[10,0],[11,0],[12,0],[13,0],[14,0],
                                     [15,0],[16,0],[17,0],[18,6],[19,10],[20,10],[21,0], ["NoData", 10]]))

In [30]:
# change 0s to 1
Corine_raster_cost = Con(Corine_raster_cost == 0, 1, Corine_raster_cost)

In [31]:
Corine_raster_cost.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\Corine_raster_cost")

#### Cost for roads
Erase urban areas from roads multiline to just have roads outside of urban areas since urban areas already have a negative effect

In [1]:
# create buffer around roads
Roads_buffer50 = arcpy.Buffer_analysis("atkis_road_Clip1", "Roads_buffer50", "50 Meters", "FULL", 
                      "ROUND", "ALL")

In [ ]:
# Corine land cover polygons --> select by attribute manually --> urban areas, industry, dump sites, Sport and leisure facilities, construction sites
# export to polygon
# erase those from roads_buffer (erase tool)

In [2]:
# convert to Raster
Roads50_raster = arcpy.PolygonToRaster_conversion("Roads_buffer50_Erase", "Shape_Area", 
                                 "Roads50_raster", "MAXIMUM_AREA", cellsize = 5)

In [3]:
# Road cost = 3, NoData = 1
Roads50_weight3_cost = Con(IsNull(arcpy.Raster("Roads50_raster")), 1, 3)

In [4]:
arcpy.Raster("Roads50_weight3_cost").save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\Roads50_weight3_cost")

In [6]:
# Adding roads cost to Corine costs
Corine_roads3_cost = Con(arcpy.Raster("Corine_raster_cost") == 1, arcpy.Raster("Roads50_weight3_cost"), arcpy.Raster("Corine_raster_cost"))

In [11]:
Corine_roads3_cost.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\Corine_roads3_cost")

#### Comined cost
Avoidance of human activities more important than visibility

In [10]:
Combined_r3_cost0406 = 0.4 * arcpy.Raster("Vis_cost_x9p1") + 0.6 * arcpy.Raster("Corine_roads3_cost")

In [36]:
Combined_r3_cost0406.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\Combined_r3_cost0406")

In [41]:
# Trying more extreme cost raster. power of 2
Combined_r3_cost0406p2 = Combined_r3_cost0406 ** 2

In [42]:
Combined_r3_cost0406p2.save(r"D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\Combined_r3_cost0406p2")

# Cost Distance
Distance Accumulation, Optimal Path, Barrier

In [ ]:
# Creating barrier manually with Corine LC (water bodies)
# Creating Core forest manually with Buffer, Erase, ... tools
    # identify small vegetated gaps in forest and declare them as forest
    # dissolve forest, add 300 meter ring outside of AOI to not cut off the 
        # outer border of the forests bc of being at the edge of the AOI 
    # cutting of 300m buffer to get core forest
    # Multipart to Singlepart forest areas to get individual forests
    # Multipart to Singlepart vineyard areas to get inidividual vineyards

In [56]:
# Adding amount of boars
# 250 Tiere pro 5000 Hektar/10000m, (consideration from Brauchler)
arcpy.CalculateField_management("Wald_Kern300_single", "ExpectedBoars", 
                                'math.ceil(!Shape_Area! / 10000 / 5000 * 250)', "PYTHON3")

<Result 'Wald_Kern300_single'>

In [60]:
# field for example points
arcpy.CalculateField_management("Wald_Kern300_single", "ExamplePoints", 
                                'math.ceil((!ExpectedBoars! - 1) /20)', "PYTHON3")

<Result 'Wald_Kern300_single'>

In [46]:
# Adding a size factor to the core forests bc there are potentially more boars
# SizeStart 
arcpy.CalculateField_management("Wald_Kern300_single", "SizeStart", 
                                '!ExpectedBoars! * (-200) ', "PYTHON3")

<Result 'Wald_Kern300_single'>

In [38]:
# SizeFactor
arcpy.CalculateField_management("Wald_Kern300_single", "SizeFactor", 
                                '(1 / (!ExpectedBoars!) + 1)**5', "PYTHON3")

<Result 'Wald_Kern300_single'>

#### Points for boars

In [51]:
arcpy.CreateRandomPoints_management("", "samplepoints_ExpectedBoars", 
                                    "Wald_Kern300_single", number_of_points_or_field = "ExpectedBoars", minimum_allowed_distance = "150 Meters")

<Result 'D:\\Geoinformatik\\Advanced GIS\\AdvGIS term paper\\AdvGIS term paper.gdb\\samplepoints_ExpectedBoars'>

In [61]:
# example points to show examplatory paths to show how the cost raster influences the modell
arcpy.CreateRandomPoints_management("", "example_points", "Wald_Kern300_single", 
                                    number_of_points_or_field = "ExamplePoints", minimum_allowed_distance = "1500 Meters")

<Result 'D:\\Geoinformatik\\Advanced GIS\\AdvGIS term paper\\AdvGIS term paper.gdb\\example_points'>

#### Cleaning Vineyard polygons

In [ ]:
# Delete very small polygons (artifacts from digitization)
# visually inspected how large the correct polygons are
# selected areas larger than 1000m2 and exported those 

#### Parameters for Path Distance

In [ ]:
# Path Distance Parameters for PathDis_Wald4:
# cost raster: Combined_r3_cost0406p2
# Input surface raster: DEM
# Accumulative cost resistance raster: 0.002
# Start Cost: Forest StartSize

In [3]:
# Second Path distance calculation to show result without start size
PathDis_ohneStartSize = PathDistance("Wald_Kern300_single", "Combined_r3_cost0406p2", "DEM_5m_update.tif",
                     source_resistance_rate = 0.002)

### Beispielhafte Routen von Wildschweinen

In [2]:
# creating list
points_list = []


In [4]:
# filling list with point layer names
pointLayer_names = ["TTSample13", "TTSample10", "TTSample8", "TTSample7", "TTSample6", "TTSample2","TTSample1"]
print(pointLayer_names)

['TTSample13', 'TTSample10', 'TTSample8', 'TTSample7', 'TTSample6', 'TTSample2', 'TTSample1']


In [6]:
# create back raster variable to temporarily store back direction raster
back = arcpy.Raster("back")

In [7]:
# compute distAcc and opiPaths
for point in pointLayer_names:
    dist_acc = DistanceAccumulation(point, in_surface_raster = "DEM_5m_update.tif", 
                                    in_cost_raster = "Combined_r3_cost0406p2", out_back_direction_raster = back)
    OptimalPathAsLine("Vineyards_aggr_sample", dist_acc, back, point + "_noPower")
    print(point + " done")



TTSample13 done
TTSample10 done
TTSample8 done
TTSample7 done
TTSample6 done
TTSample2 done
TTSample1 done
